In [1]:
import requests
import pandas as pd
import json
from datetime import datetime
import folium
from folium.plugins import HeatMap
import streamlit as st

# Carrega o arquivo JSON
list_city = r"C:\Users\sensix\Desktop\PESSOAL\PESSOAL\PORTIFOLIO DATA SCIENCE\pd\pd\Dados\city_coordinates.json"
with open(list_city, 'r') as file:
    city_coordinates = json.load(file)

# Token da API OpenWeatherMap
token = '11f2871a588e33e781d6365014dba754'

# Lista para armazenar os dados de cada cidade
data_list = []

# Exibir mensagem de aviso
st.warning("Carregando dados... Isso pode levar um momento.")

# Inicia o loop sobre as chaves do dicionário city_coordinates
for city, coordinates in city_coordinates.items():
    lat = coordinates["latitude"]
    lon = coordinates["longitude"]
    
    # Faz a solicitação para a API do OpenWeatherMap
    url = f'http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={token}'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        aqi = data["list"][0]["main"]
        components = data["list"][0]["components"]
        dt = datetime.fromtimestamp(data["list"][0]["dt"])
        
        # Adiciona os dados à lista
        data_list.append({
            "City": city,
            "Latitude": lat,
            "Longitude": lon,
            "Timestamp": dt,
            "AQI": aqi.get("aqi",None),
            "CO": components.get("co", None),
            "NO": components.get("no", None),
            "NO2": components.get("no2", None),
            "O3": components.get("o3", None),
            "SO2": components.get("so2", None),
            "PM2.5": components.get("pm2_5", None),
            "PM10": components.get("pm10", None),
            "NH3": components.get("nh3", None)

        })
    else:
        print(f"Falha na solicitação para {city}")

# Cria o DataFrame
df_final = pd.DataFrame(data_list)

# Criar um mapa centrado no Brasil
mapa = folium.Map(location=[-14.235, -51.9253], zoom_start=4)

# Converter os dados de AQI para uma lista de tuplas (latitude, longitude, AQI)
dados_aqi = df_final[['Latitude', 'Longitude', 'AQI']].dropna()
dados_aqi = dados_aqi.values.tolist()

# Adicionar o mapa de calor ao mapa
folium.plugins.HeatMap(dados_aqi, gradient={.2: '#2b8318', .4: '#abdda4', .6: '#ffffbf', .8: '#fdae61', .1: '#d7191c'},radius = 20,blur=15).add_to(mapa)

texto_legenda = '''
                <div style="position: fixed; bottom: 50px; left: 50px; background-color: rgba(255, 255, 255,0.45);
                border-radius: 5px; padding: 10px; z-index: 1000; font-size: 14px;">
                <p><strong>Legenda:</strong></p>
                <p><span style="color: #2b8318;">Bom: 1</span> </p>
                <p><span style="color: #abdda4;">Razóavel: 2</span> </p>
                <p><span style="color: #ffffbf;">Moderado: 3</span> </p>
                <p><span style="color: #fdae61;">Ruim: 4</span> </p>
                <p><span style="color: #d7191c;">Muito Ruim: 5</span> </p>
                </div>
                '''

mapa.get_root().html.add_child(folium.Element(texto_legenda))

# Exibir o mapa
st.write(mapa)


2024-03-10 15:50:36.481 
  command:

    streamlit run c:\Users\sensix\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


KeyboardInterrupt: 